In [1]:
import pickle
import time
from selenium import webdriver

driver = webdriver.Chrome()

def guardar_cookies_ig():
    driver.get("https://www.instagram.com/accounts/login/")
    print("Por favor, logueate manualmente en Instagram...")
    # Instagram suele pedir código al correo o celular, tómate tu tiempo.
    time.sleep(60) 
    
    with open("instagram_cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)
    
    print("¡Cookies de Instagram guardadas con éxito!")
    driver.quit()

guardar_cookies_ig()

Por favor, logueate manualmente en Instagram...
¡Cookies de Instagram guardadas con éxito!


In [2]:
import pickle
import time
import random
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

def configurar_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications")
    # Sigilo para Instagram
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    })
    return driver

driver = configurar_driver()

def esta_logueado():
    """Verifica si estamos dentro buscando el icono de 'Direct' o el perfil"""
    try:
        # Buscamos el SVG de mensajes o la barra lateral
        driver.find_element(By.XPATH, "//*[@aria-label='Inicio' or @aria-label='Perfil']")
        return True
    except:
        return False

def cargar_cookies_ig():
    print("Intentando cargar sesión de Instagram...")
    driver.get("https://www.instagram.com")
    time.sleep(5)
    try:
        with open("instagram_cookies.pkl", "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                if 'expiry' in cookie: del cookie['expiry']
                driver.add_cookie(cookie)
        driver.refresh()
        time.sleep(6)
        return esta_logueado()
    except:
        return False

def realizar_scraping_ig(tema):
    intentos = 0
    max_intentos = 2
    
    logueado = cargar_cookies_ig()
    
    while not logueado:
        intentos += 1
        if intentos < max_intentos:
            print(f"❌ Intento {intentos} fallido. Reintentando en 1 minuto...")
            time.sleep(60)
            logueado = cargar_cookies_ig()
        else:
            print("\nSISTEMA BLOQUEADO: Entra MANUALMENTE a Instagram en el navegador.")
            while not esta_logueado():
                time.sleep(5)
            # Guardamos las nuevas cookies para que no falle mañana
            with open("instagram_cookies.pkl", "wb") as file:
                pickle.dump(driver.get_cookies(), file)
            logueado = True

    # 1. Búsqueda por Hashtag o Palabra clave
    # Nota: Instagram maneja mejor las búsquedas por URL directa en scraping
    print(f"Buscando: {tema}...")
    search_url = f"https://www.instagram.com/explore/tags/{tema.replace(' ', '')}/"
    driver.get(search_url)
    time.sleep(8)

    # 2. Extracción de pies de foto (Captions) - Anonimizado
    with open('comentarios_ig_anonimos.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Texto_Publicacion"])

        for i in range(4):
            print(f"Scroll {i+1}...")
            # Buscamos los links de las fotos para obtener el texto
            # En Instagram, el texto de la publicación suele estar en alt de las imágenes o en un span específico
            post_texts = driver.find_elements(By.TAG_NAME, "img")
            
            for p in post_texts:
                try:
                    texto = p.get_attribute("alt")
                    # Filtramos solo si el texto es largo y parece una descripción
                    if texto and len(texto) > 30 and "Foto de" not in texto:
                        writer.writerow([texto.replace('\n', ' ')])
                        print(f"  [+] Extraído: {texto[:50]}...")
                except:
                    continue

            driver.execute_script("window.scrollBy(0, 1200);")
            time.sleep(random.uniform(5, 7))

    print("\n✅ Proceso completado en Instagram.")
    driver.quit()

if __name__ == "__main__":
    # En Instagram es mejor usar una sola palabra o hashtag pegado
    realizar_scraping_ig("inteligenciaartificial")

Intentando cargar sesión de Instagram...
Buscando: inteligenciaartificial...
Scroll 1...
  [+] Extraído: Photo by Luis Cué | Asesor financiero on November ...
  [+] Extraído: Photo by Diseño de Páginas Web & Tiendas Online in...
  [+] Extraído: Photo by Dr. José Luis Soto Noriega - Mastologo (C...
  [+] Extraído: 🔥 FREE vs PAID en Inteligencia Artificial

Aquí ti...
  [+] Extraído: Photo by Conatel Venezuela on December 09, 2025. M...
  [+] Extraído: Photo by El_diario_de_Fany on January 16, 2026. Ma...
  [+] Extraído: Photo by Teorema-Infotep on August 28, 2025....
  [+] Extraído: Photo by 0223com on January 20, 2026. May be a Twi...
  [+] Extraído: Photo by CENTECPRO CARACAS on December 03, 2025. M...
  [+] Extraído: Photo by BdB ANALPA on January 19, 2026. May be an...
  [+] Extraído: Photo by Federación Mujeres Lorca on December 10, ...
  [+] Extraído: Photo by TDN in Tucuman, Argentina. May be an imag...
  [+] Extraído: Photo by Antonio Serrano Acitores in Universidad R...
  [+] E